# Sentiment Analysis

### Twitter Terms

April 2019

Based on original script [here](https://github.com/cipher813/bitcoin_prediction/blob/master/scripts/tweethack1.py).

In [1]:
import re
import tweepy
import pandas as pd
from textblob import TextBlob
from collections import defaultdict

In [2]:
class TwitterClient:
    def __init__(self):
        self.consumer_key = %env TWITTER_API_KEY
        self.consumer_secret = %env TWITTER_API_SECRET_KEY
        self.access_token = %env TWITTER_ACCESS_TOKEN
        self.access_secret = %env TWITTER_ACCESS_TOKEN_SECRET

        self.auth = tweepy.OAuthHandler(self.consumer_key, self.consumer_secret)
        self.auth.set_access_token(self.access_token, self.access_secret)

        self.api = tweepy.API(self.auth)
        
    def get_tweets(self, term):
        """Get the latest set of tweets by term, grouped by sentiment.
        
        :term: str Term to match in tweets
        """
        td = defaultdict(list)
        tweets = self.api.search(term,tweet_mode="extended")
        for tweet in tweets:
            if 'retweeted_status' in tweet._json:
                txt = TextBlob(tweet._json['retweeted_status']['full_text'])
            else:
                txt = TextBlob(tweet.full_text)
            polarity = txt.sentiment.polarity
            if polarity > 0:
                td['pos'].append(f"{txt} ({polarity:.2f})")
            elif polarity < 0:
                td['neg'].append(f"{txt} ({polarity:.2f})")
            else:
                td['neu'].append(f"{txt}-{polarity:.2f}")
        pos, neg, neu = len(td['pos']),len(td['neg']),len(td['neu'])
        tot = pos+neg+neu
        print(f"Tweet stats:\nPositive:{pos}, {pos/tot*100:.1f}%\nNegative:{neg}, {neg/tot*100:.1f}%\nNeutral:{neu}, {neu/tot*100:.1f}%")
        return td
    
    def clean_tweet(self, tweet):
        """Format tweet into clean string.
        
        :tweet: str tweet to clean
        """
        tweet = ' '.join(re.sub(r"(.+@[A-Za-z0-9_]+)|([^0-9A-Za-z\.\s\t]) |(\w+:\/\/\S+)", "", tweet).split()) 
        return tweet

In [3]:
twitter = TwitterClient()
d = twitter.get_tweets("Trump")

Tweet stats:
Positive:3, 27.3%
Negative:5, 45.5%
Neutral:3, 27.3%


In [4]:
tweet = d['pos'][0]
tweet

"In 2005, many of us who'd voted for George W. Bush the year before opposed his nomination of Harriet Miers to the Supreme Court, and we prevailed. Which Trump supporters will speak up against the (perhaps more egregious) nominations of Stephen Moore and Herman Cain to the Fed? (0.50)"

In [5]:
tweet = twitter.clean_tweet(d['pos'][0])
tweet

"In 2005many of us who'd voted for George W. Bush the year before opposed his nomination of Harriet Miers to the Supreme Courtand we prevailed. Which Trump supporters will speak up against the (perhaps more egregiousnominations of Stephen Moore and Herman Cain to the Fed(0.50)"

In [6]:
d['pos']

["In 2005, many of us who'd voted for George W. Bush the year before opposed his nomination of Harriet Miers to the Supreme Court, and we prevailed. Which Trump supporters will speak up against the (perhaps more egregious) nominations of Stephen Moore and Herman Cain to the Fed? (0.50)",
 "@cenkuygur @AOC I'd say Trump has got her beat by a mile. Until they go after her with no evidence &amp; try to destroy her entire family &amp; put them in prison for winning an election Chunk Yanker, she's no victim just a dolt.\nDude, F**g really. \nTrump bad, Sandra Casio Ortiz good? \nGigs up Cenk.. https://t.co/QBnS9YztYH (0.08)",
 'There are a heck of a lot more jobs fighting climate change than there are in denying climate change. #OurClimateMoment https://t.co/HqHDtKIOVe (0.50)']

In [7]:
d['neg']

['Looks like Bob Mueller’s team of 13 Trump Haters &amp; Angry Democrats are illegally leaking information to the press while the Fake News Media make up their own stories with or without sources - sources no longer matter to our corrupt &amp; dishonest Mainstream Media, they are a Joke! (-0.30)',
 'The hate is coming frim the left. The Republicans never treated Obama the way the Democrats treat President Trump https://t.co/BaDJtFrB3Z (-0.40)',
 '"They dislike him with a passion, and they want his tax returns to destroy him." - @ChuckGrassley\n\nNope, sorry, Chuck. Every president since Nixon has shown the American people their tax returns.\n\n#CongressCanRequest\n\nhttps://t.co/FaybXK4jII (-0.23)',
 'Ilhan Omar was labeled an anti-Semite for merely suggesting what Trump said out loud. https://t.co/dNiUFra80S (-0.20)',
 'trump literally gets up and says “you people love money and your country, which is israel” every time he talks to jews it’s beyond disgusting https://t.co/03qK3LVmKE (

In [8]:
d['neu']

['Far-left activists are recirculating President Trump\'s "these aren\'t people, they\'re animals" quote.\n\nSo what are we calling savage MS-13 gang members now? Humanitarians? What we should be asking is why Democrats are not calling them \'animals.\'-0.00',
 'Here are photos of Donald, Ivanka and Eric Trump doing business in Doha, Qatar in 2008. Ivanka used to sell her jewelry there and bragged about her relationship with the government, Don Jr and Eric have been there a lot and Cohen reportedly tried to get $1M from the Gov’t. https://t.co/FAi5mqMEha-0.00',
 '1) Expertos en inteligencia escriben directamente a Trump: "No defendemos a Maduro.. pero subrayamos que muchos de los problemas de Venezuela han sido exacerbados por las políticas estadounidenses y sus intentos de derrocar al presidente"\nhttps://t.co/0didwpeUOi-0.00']